# Heterogeneous Graph Learning
https://pytorch-geometric.readthedocs.io/en/latest/tutorial/heterogeneous.html

In [1]:
import os
import torch

In [2]:
#colabでやるならこっち
#os.environ['TORCH'] = torch.__version__
#print(torch.__version__)

#!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
#!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
#!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
#!pip install graphviz
#!pip install torchviz

In [3]:
#ローカルのcondaでやるならこっち
#%conda install python==3.10.6
#%conda install -c conda-forge matplotlib -y
#%conda install -c pytorch pytorch torchaudio torchvision -y
#%conda install -c pytorch pytorch torchvision torchaudio -y
#%conda install -c anaconda networkx -y
#%conda install -c conda-forge python-graphviz -y
#%conda install -c conda-forge graphviz -y
#%pip install torchviz 

In [4]:
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

1.12.1


In [5]:
#%pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
#%pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
#%pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

In [6]:
#%pip install torch_geometric #上のセルでうまくいかない場合は拡張パッケージのインストールはあきらめる

## HeteroDataの概要
![](https://pytorch-geometric.readthedocs.io/en/latest/_images/hg_example.svg)

In [7]:
from torch_geometric.datasets import OGB_MAG

dataset = OGB_MAG(root='./data', preprocess='metapath2vec')
data = dataset[0]

c:\Users\NEC-PCuser\anaconda3\envs\torch_env\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [8]:
data

HeteroData(
  paper={
    x=[736389, 128],
    year=[736389],
    y=[736389],
    train_mask=[736389],
    val_mask=[736389],
    test_mask=[736389],
  },
  author={ x=[1134649, 128] },
  institution={ x=[8740, 128] },
  field_of_study={ x=[59965, 128] },
  (author, affiliated_with, institution)={ edge_index=[2, 1043998] },
  (author, writes, paper)={ edge_index=[2, 7145660] },
  (paper, cites, paper)={ edge_index=[2, 5416271] },
  (paper, has_topic, field_of_study)={ edge_index=[2, 7505078] }
)

もしオリジナルのデータセットを構成する場合は以下のように書いていく
```python

from torch_geometric.data import HeteroData

data = HeteroData()

data['paper'].x = ... # [num_papers, num_features_paper]
data['author'].x = ... # [num_authors, num_features_author]
data['institution'].x = ... # [num_institutions, num_features_institution]
data['field_of_study'].x = ... # [num_field, num_features_field]

data['paper', 'cites', 'paper'].edge_index = ... # [2, num_edges_cites]
data['author', 'writes', 'paper'].edge_index = ... # [2, num_edges_writes]
data['author', 'affiliated_with', 'institution'].edge_index = ... # [2, num_edges_affiliated]
data['paper', 'has_topic', 'field_of_study'].edge_index = ... # [2, num_edges_topic]

data['paper', 'cites', 'paper'].edge_attr = ... # [num_edges_cites, num_features_cites]
data['author', 'writes', 'paper'].edge_attr = ... # [num_edges_writes, num_features_writes]
data['author', 'affiliated_with', 'institution'].edge_attr = ... # [num_edges_affiliated, num_features_affiliated]
data['paper', 'has_topic', 'field_of_study'].edge_attr = ... # [num_edges_topic, num_features_topic]

```

In [9]:
paper_node_data = data['paper']
cites_edge_data = data['paper', 'cites', 'paper']

In [10]:
paper_node_data,cites_edge_data

({'x': tensor([[-0.0954,  0.0408, -0.2109,  ...,  0.0616, -0.0277, -0.1338],
         [-0.1510, -0.1073, -0.2220,  ...,  0.3458, -0.0277, -0.2185],
         [-0.1148, -0.1760, -0.2606,  ...,  0.1731, -0.1564, -0.2780],
         ...,
         [ 0.0228, -0.0865,  0.0981,  ..., -0.0547, -0.2077, -0.2305],
         [-0.2891, -0.2029, -0.1525,  ...,  0.1042,  0.2041, -0.3528],
         [-0.0890, -0.0348, -0.2642,  ...,  0.2601, -0.0875, -0.5171]]), 'year': tensor([2015, 2012, 2012,  ..., 2016, 2017, 2014]), 'y': tensor([246, 131, 189,  ..., 266, 289,   1]), 'train_mask': tensor([True, True, True,  ..., True, True, True]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'test_mask': tensor([False, False, False,  ..., False, False, False])},
 {'edge_index': tensor([[     0,      0,      0,  ..., 736388, 736388, 736388],
         [    88,  27449, 121051,  ..., 421711, 427339, 439864]])})

In [11]:
cites_edge_data = data['paper', 'paper']
print(cites_edge_data)
cites_edge_data = data['cites']
print(cites_edge_data)

{'edge_index': tensor([[     0,      0,      0,  ..., 736388, 736388, 736388],
        [    88,  27449, 121051,  ..., 421711, 427339, 439864]])}
{'edge_index': tensor([[     0,      0,      0,  ..., 736388, 736388, 736388],
        [    88,  27449, 121051,  ..., 421711, 427339, 439864]])}


新しいノードを増やしたり減らしたりする場合
```python
data['paper'].year = ...    # Setting a new paper attribute
del data['field_of_study']  # Deleting 'field_of_study' node type
del data['has_topic']       # Deleting 'has_topic' edge type
```

In [12]:
# メタデータへのアクセス
node_types, edge_types = data.metadata()
print(node_types)
print(edge_types)

['paper', 'author', 'institution', 'field_of_study']
[('author', 'affiliated_with', 'institution'), ('author', 'writes', 'paper'), ('paper', 'cites', 'paper'), ('paper', 'has_topic', 'field_of_study')]


(**A**,**B**,**C**)なら、**A**が**C**を**B**したということを示している

In [13]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cpu device


In [14]:
data = data.to(device)

In [15]:
data.has_isolated_nodes() #孤立しているノードの有無

False

In [16]:
data.has_self_loops() #セルフループの有無

False

In [17]:
data.is_undirected() #接続しているノードの有無

False

## Heterogeneous Graph Transformations

In [18]:
import torch_geometric.transforms as T

data = T.ToUndirected()(data)
data = T.AddSelfLoops()(data)
data = T.NormalizeFeatures()(data)
data

HeteroData(
  paper={
    x=[736389, 128],
    year=[736389],
    y=[736389],
    train_mask=[736389],
    val_mask=[736389],
    test_mask=[736389],
  },
  author={ x=[1134649, 128] },
  institution={ x=[8740, 128] },
  field_of_study={ x=[59965, 128] },
  (author, affiliated_with, institution)={ edge_index=[2, 1043998] },
  (author, writes, paper)={ edge_index=[2, 7145660] },
  (paper, cites, paper)={ edge_index=[2, 11529061] },
  (paper, has_topic, field_of_study)={ edge_index=[2, 7505078] },
  (institution, rev_affiliated_with, author)={ edge_index=[2, 1043998] },
  (paper, rev_writes, author)={ edge_index=[2, 7145660] },
  (field_of_study, rev_has_topic, paper)={ edge_index=[2, 7505078] }
)

## How to construct Heterogeneous GNN models

### Automatically Converting GNN Models

In [19]:
from torch_geometric.nn import SAGEConv
import torch.nn.functional as F

class GNN(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

In [20]:
import torch_geometric.transforms as T
from torch_geometric.nn import to_hetero

model = GNN(hidden_channels=64, out_channels=dataset.num_classes)
model = to_hetero(model, data.metadata(), aggr='sum')

In [21]:
model

GraphModule(
  (conv1): ModuleDict(
    (author__affiliated_with__institution): SAGEConv((-1, -1), 64, aggr=mean)
    (author__writes__paper): SAGEConv((-1, -1), 64, aggr=mean)
    (paper__cites__paper): SAGEConv((-1, -1), 64, aggr=mean)
    (paper__has_topic__field_of_study): SAGEConv((-1, -1), 64, aggr=mean)
    (institution__rev_affiliated_with__author): SAGEConv((-1, -1), 64, aggr=mean)
    (paper__rev_writes__author): SAGEConv((-1, -1), 64, aggr=mean)
    (field_of_study__rev_has_topic__paper): SAGEConv((-1, -1), 64, aggr=mean)
  )
  (conv2): ModuleDict(
    (author__affiliated_with__institution): SAGEConv((-1, -1), 349, aggr=mean)
    (author__writes__paper): SAGEConv((-1, -1), 349, aggr=mean)
    (paper__cites__paper): SAGEConv((-1, -1), 349, aggr=mean)
    (paper__has_topic__field_of_study): SAGEConv((-1, -1), 349, aggr=mean)
    (institution__rev_affiliated_with__author): SAGEConv((-1, -1), 349, aggr=mean)
    (paper__rev_writes__author): SAGEConv((-1, -1), 349, aggr=mean)
   

`torch_geometric.nn.to_hetero()`もしくは`torch_geometric.nn.to_hetero_with_bases()`を使うとmetadataを基に勝手にheteroなモデルにしてくれる

![](https://pytorch-geometric.readthedocs.io/en/latest/_images/to_hetero.svg)

入力特徴の数とテンソルのサイズはタイプによって異なるため、PyGは遅延初期化を利用して、heterogeneous GNNsのパラメーターを初期化できます (in_channels 引数として -1 で示されます)。

これにより、計算グラフのすべてのテンソル サイズを計算して追跡する必要がなくなります。遅延初期化は、既存のすべての PyG 演算子でサポートされています。モデルのパラメータを 1 回呼び出すことで初期化できます。

In [22]:
with torch.no_grad():  # Initialize lazy modules.
    out = model(data.x_dict, data.edge_index_dict)

In [23]:
import  torch.nn.functional as F
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)


def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x_dict, data.edge_index_dict)
    mask = data['paper'].train_mask #valid, testのときはそれぞれ異なるmaskを用いる
    loss = F.cross_entropy(out['paper'][mask], data['paper'].y[mask])
    loss.backward()
    optimizer.step()
    return float(loss)

In [25]:
#多分こんな感じでTrainingすればよき
#n_epoch = 5

#for i in range(n_epoch):
#    loss = train()
#    print(f"Epoch {i} Train Loss={loss}")

### Using Heterogeneous Convolution Wrapper

`torch_geometric.nn.conv.HeteroConv` を使用すると、カスタム異種メッセージを定義し、異種グラフ用の任意の MP-GNN を最初から構築する関数を更新できます。自動コンバータ `to_hetero()` はすべてのエッジ タイプに同じ演算子を使用しますが、ラッパーでは異なるエッジ タイプに異なる演算子を定義できます。ここで、HeteroConv は、グラフ データ内のエッジ タイプごとに 1 つずつ、サブモジュールの辞書を入力として受け取ります。次の例は、それを適用する方法を示しています。

In [29]:
import torch_geometric.transforms as T
from torch_geometric.datasets import OGB_MAG
from torch_geometric.nn import HeteroConv, GCNConv, SAGEConv, GATConv, Linear

dataset = OGB_MAG(root='./data', preprocess='metapath2vec', transform=T.ToUndirected())
data = dataset[0]

class HeteroGNN(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels, num_layers):
        super().__init__()

        self.convs = torch.nn.ModuleList()
        for _ in range(num_layers):
            conv = HeteroConv({
                ('paper', 'cites', 'paper'): GCNConv(-1, hidden_channels),
                ('author', 'writes', 'paper'): SAGEConv((-1, -1), hidden_channels),
                ('paper', 'rev_writes', 'author'): GATConv((-1, -1), hidden_channels, add_self_loops=False),
            }, aggr='sum')
            self.convs.append(conv)

        self.lin = Linear(hidden_channels, out_channels)

    def forward(self, x_dict, edge_index_dict):
        for conv in self.convs:
            x_dict = conv(x_dict, edge_index_dict)
            x_dict = {key: x.relu() for key, x in x_dict.items()}
        return self.lin(x_dict['author'])

model = HeteroGNN(hidden_channels=64, out_channels=dataset.num_classes, num_layers=2)

In [30]:
with torch.no_grad():  # Initialize lazy modules.
     out = model(data.x_dict, data.edge_index_dict)

### Deploy Existing Heterogeneous Operators

PyG provides operators (e.g., torch_geometric.nn.conv.HGTConv), which are specifically designed for heterogeneous graphs. These operators can be directly used to build heterogeneous GNN models as can be seen in the following example:

In [31]:
import torch_geometric.transforms as T
from torch_geometric.datasets import OGB_MAG
from torch_geometric.nn import HGTConv, Linear

dataset = OGB_MAG(root='./data', preprocess='metapath2vec', transform=T.ToUndirected())
data = dataset[0]

class HGT(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels, num_heads, num_layers):
        super().__init__()

        self.lin_dict = torch.nn.ModuleDict()
        for node_type in data.node_types:
            self.lin_dict[node_type] = Linear(-1, hidden_channels) #node_typeごとに異なる次元 -> hidden_channels次元へ

        self.convs = torch.nn.ModuleList()
        for _ in range(num_layers):
            conv = HGTConv(hidden_channels, hidden_channels, data.metadata(),
                           num_heads, group='sum')
            self.convs.append(conv)

        self.lin = Linear(hidden_channels, out_channels)

    def forward(self, x_dict, edge_index_dict):
        for node_type, x in x_dict.items():
            x_dict[node_type] = self.lin_dict[node_type](x).relu_()

        for conv in self.convs:
            x_dict = conv(x_dict, edge_index_dict)

        return self.lin(x_dict['author'])

model = HGT(hidden_channels=64, out_channels=dataset.num_classes, num_heads=2, num_layers=2)

In [ ]:
with torch.no_grad():  # Initialize lazy modules.
     out = model(data.x_dict, data.edge_index_dict)

## Heterogeneous Graph Samplers

PyG provides various functionalities for sampling heterogeneous graphs, i.e. in the standard torch_geometric.loader.NeighborLoader class or in dedicated heterogeneous graph samplers such as torch_geometric.loader.HGTLoader. This is especially useful for efficient representation learning on large heterogeneous graphs, where processing the full number of neighbors is too computationally expensive. Heterogeneous graph support for other samplers such as torch_geometric.loader.ClusterLoader or torch_geometric.loader.GraphSAINTLoader will be added soon. Overall, all heterogeneous graph loaders will produce a HeteroData object as output, holding a subset of the original data, and mainly differ in the way their sampling procedures works. As such, only minimal code changes are required to convert the training procedure from full-batch training to mini-batch training.

Performing neighbor sampling using NeighborLoader works as outlined in the following example:

In [33]:
import torch_geometric.transforms as T
from torch_geometric.datasets import OGB_MAG
from torch_geometric.loader import NeighborLoader

transform = T.ToUndirected()  # Add reverse edge types.
data = OGB_MAG(root='./data', preprocess='metapath2vec', transform=transform)[0]

train_loader = NeighborLoader(
    data,
    # Sample 15 neighbors for each node and each edge type for 2 iterations:
    num_neighbors=[15] * 2,
    # Use a batch size of 128 for sampling training nodes of type "paper":
    batch_size=128,
    input_nodes=('paper', data['paper'].train_mask),
)

batch = next(iter(train_loader))

ImportError: 'NeighborSampler' requires either 'pyg-lib' or 'torch-sparse'

Notably, NeighborLoader works for both homogeneous and heterogeneous graphs. When operating in heterogeneous graphs, more fine-grained control over the amount of sampled neighbors of individual edge types is possible, but not necessary, e.g.:

In [ ]:
num_neighbors = {key: [15] * 2 for key in data.edge_types}

In [ ]:
batch

In [ ]:
def train():
    model.train()

    total_examples = total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        batch = batch.to('cuda:0')
        batch_size = batch['paper'].batch_size
        out = model(batch.x_dict, batch.edge_index_dict)
        loss = F.cross_entropy(out['paper'][:batch_size],
                               batch['paper'].y[:batch_size])
        loss.backward()
        optimizer.step()

        total_examples += batch_size
        total_loss += float(loss) * batch_size

    return total_loss / total_examples